In [89]:
import pandas as pd

# Import data

In [90]:
order = pd.read_excel('MINI-TEST (MCI SOLUTIONS - DATA ANALYST INTERN).xlsx', sheet_name='ORDERS')
product = pd.read_excel('MINI-TEST (MCI SOLUTIONS - DATA ANALYST INTERN).xlsx', sheet_name='PRODUCTS')

# EDA
## `Products` sheet

In [91]:
product.head(3)

,Product ID,Product Line,Product Category,Product Name,Supplier Country
0,210100100001,Children,Children Outdoors,Boy's and Girl's Ski Pants with Braces,NO
1,210100100002,Children,Children Outdoors,Children's Jacket,ES
2,210100100003,Children,Children Outdoors,Children's Jacket Sidney,NO


In [92]:
# check null values
product.isna().sum()

Product ID          0
Product Line        0
Product Category    0
Product Name        0
Supplier Country    0
dtype: int64

In [93]:
# check duplicates
product.duplicated().sum()

0

In [94]:
# check data types
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5504 entries, 0 to 5503
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Product ID        5504 non-null   int64 
 1   Product Line      5504 non-null   object
 2   Product Category  5504 non-null   object
 3   Product Name      5504 non-null   object
 4   Supplier Country  5504 non-null   object
dtypes: int64(1), object(4)
memory usage: 215.1+ KB


In [95]:
# check typos for each column
for col in product.columns[1:]:
    print(product[col].unique())

['Children' 'Clothes & Shoes' 'Outdoors' 'Sports']
['Children Outdoors' 'Children Sports' 'Clothes' 'Shoes' 'Outdoors'
 'Assorted Sports Articles' 'Golf' 'Indoor Sports' 'Racket Sports'
 'Running - Jogging' 'Swim Sports' 'Team Sports' 'Winter Sports']
["Boy's and Girl's Ski Pants with Braces" "Children's Jacket"
 "Children's Jacket Sidney" ... 'Top Equipe 99 Black'
 'Alpine Ski Bag 2-pair Black/Yellow' 'Alpine Ski Bag Black/Yellow']
['NO' 'ES' 'US' 'SE' 'FR' 'GB' 'CA' 'NL' 'DK' 'AU' 'PT' 'DE' 'BE']


***Comment: No issue detected in `products` sheet***
## `Orders` sheet
### Check

In [96]:
order.head(3)

,Customer ID,Customer Status,Date Order was placed,Delivery Date,Order ID,Product ID,Quantity Ordered,Total Retail Price for This Order,Cost Price Per Unit
0,579,Silver,2017-01-01,2017-01-07,123002578,220101400106,2,92.6,20.70
1,7574,SILVER,2017-01-01,2017-01-05,123004074,210201000009,1,21.7,9.95
2,28861,Gold,2017-01-01,2017-01-04,123000871,230100500068,1,1.7,0.80


In [97]:
# check null values
order.isna().sum()

Customer ID                          0
Customer Status                      0
Date Order was placed                0
Delivery Date                        0
Order ID                             0
Product ID                           0
Quantity Ordered                     0
Total Retail Price for This Order    0
Cost Price Per Unit                  0
dtype: int64

In [98]:
# check duplicates
order.duplicated().sum()

0

In [99]:
# check data types
order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Customer ID                        14999 non-null  int64         
 1   Customer Status                    14999 non-null  object        
 2   Date Order was placed              14999 non-null  datetime64[ns]
 3   Delivery Date                      14999 non-null  datetime64[ns]
 4   Order ID                           14999 non-null  int64         
 5   Product ID                         14999 non-null  int64         
 6   Quantity Ordered                   14999 non-null  int64         
 7   Total Retail Price for This Order  14999 non-null  float64       
 8   Cost Price Per Unit                14999 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int64(4), object(1)
memory usage: 1.0+ MB


In [100]:
order.groupby(['Order ID'])['Product ID'].count().sort_values()

Order ID
123000000    1
123068005    1
123068011    1
123068015    1
123068025    1
            ..
123033902    1
123033916    1
123033924    1
123033853    1
123123070    1
Name: Product ID, Length: 14999, dtype: int64

***Comment:***
* No null and duplicated values
* `Customer Status` column needs revision in terms of capitalization
* Some columns should be renamed
* As each order only has one product, so $Price Per Unit = Total Retail Price For This Order / Quantity$
* Add time to deliver column as the subtraction of Order and Delivery date
### Process

In [101]:
# Capitalize
order['Customer Status'] = order['Customer Status'].str.capitalize()
order['Customer Status'].unique()

array(['Silver', 'Gold', 'Platinum'], dtype=object)

In [102]:
# Rename column
order.rename({'Total Retail Price for This Order': 'Total Revenue',
              'Cost Price Per Unit': 'Cost per unit',
              'Date Order was placed': 'Order Date'}
             , axis=1
             , inplace=True)

order.head(3)

,Customer ID,Customer Status,Order Date,Delivery Date,Order ID,Product ID,Quantity Ordered,Total Revenue,Cost per unit
0,579,Silver,2017-01-01,2017-01-07,123002578,220101400106,2,92.6,20.70
1,7574,Silver,2017-01-01,2017-01-05,123004074,210201000009,1,21.7,9.95
2,28861,Gold,2017-01-01,2017-01-04,123000871,230100500068,1,1.7,0.80


In [103]:
order['Price per unit'] = round(order['Total Revenue']/order['Quantity Ordered'])
order['COGS'] = order['Cost per unit']*order['Quantity Ordered']
order['Total Profit'] = order['Total Revenue']-order['COGS']
order.head(3)

,Customer ID,Customer Status,Order Date,Delivery Date,Order ID,Product ID,Quantity Ordered,Total Revenue,Cost per unit,Price per unit,COGS,Total Profit
0,579,Silver,2017-01-01,2017-01-07,123002578,220101400106,2,92.6,20.70,46.0,41.40,51.20
1,7574,Silver,2017-01-01,2017-01-05,123004074,210201000009,1,21.7,9.95,22.0,9.95,11.75
2,28861,Gold,2017-01-01,2017-01-04,123000871,230100500068,1,1.7,0.80,2.0,0.80,0.90


In [104]:
# check if any discounts applied (so the prices are different from each order)
product_price = order.groupby(['Product ID'])['Price per unit'].agg(['min', 'max']).reset_index()
product_price[product_price['min'] != product_price['max']]

,Product ID,min,max


In [105]:
product_Cost = order.groupby(['Product ID'])['Cost per unit'].agg(['min', 'max']).reset_index()
product_Cost[product_Cost['min'] != product_Cost['max']]

,Product ID,min,max


***Now the data is ready***
# Export data

In [106]:
product.to_excel('Product.xlsx')
order.to_excel('Order.xlsx')